# Test Notebook für Funktionen und komplexe Analysen

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import langdetect
import re

from collections import Counter
from collections import OrderedDict
from langdetect import detect

import torch

import sys
import os
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath("../"))
import src.utilities as u
import src.plots as p
import src.constants as c
c_stopwords = c.custom_stopwords

from src.utilities import TextCleaner
from src.credentials import AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings


from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings



### Daten Laden

Die Daten, welche wir aus einer `.csv` Datei laden, welche von einer Website stammen. Mit diesen Daten möchten wir später unsere Fragen beantworten.

#### Datenbeschreibung
- **id** Unique ID des Artikels
- **title** Title des Artikels
- **date** Erscheinungsdatum des Artikels
- **author** Name des Autors
- **content** Inhalt des Artikels
- **domain** von wo der Artikel kommt
- **url** Webadresse (wie man den Artikel finden kann)

In [2]:
file_path = "../data_mc1/"
file_name_train = 'cleantech_media_dataset_v3_2024-10-28.csv'
# full_path = os.path.join(file_path, file_name_train)

column_names = ["id", "title", "date", "author", "content", "domain", "url"]
# df_train = pd.read_csv(full_path, delimiter=",", names=column_names, skiprows=1, encoding="utf-8")
df_train = pd.read_csv(file_path+file_name_train, delimiter=",", names=column_names, skiprows=1, encoding="utf-8")

Der Spalte **author ist vollständig fehlend**, nach Ansehen des Artikels ist schnell ersichtlich, dass dieser über Webscraping aus den Artikeln extrahiert werden könnte. Da aber in den Evaluierungsdaten keine Frage bezüglich des Authors vorhanden ist wird diese nun entfernt. Allerdings könnte diese Information Wert liefern im Bezug auf Ähnlichkeitsberechnung von Inhalten der Artikeln, dies könnte später erneut überprüft werden. **Vorerst wird die Spalte author erntfernt.**

In [3]:
df_train = df_train.drop(columns=["author"])

Datumsformat erstellen

In [4]:
# Datentypen korrigieren
df_train["date"] = pd.to_datetime(df_train["date"], errors="coerce")  # Falls ungültige Datumswerte vorhanden
# df_train["title"] = df_train["title"].astype(str)
# df_train["content"] = df_train["content"].astype(str)

In [5]:
def fake_list_to_text(text):
    import re
    if not isinstance(text, str):
        return text
    if text.count("',") >= 1 and "'" in text:
        parts = re.split(r"\s*,\s*", text.strip("[]"))
        parts = [p.strip(" '\"\n") for p in parts if p.strip()]
        return " ".join(parts)
    return text

In [6]:
df_train["content"] = df_train["content"].apply(fake_list_to_text)
df_train.head()

,id,title,date,content,domain,url
0,93320,"XPeng Delivered ~100,000 Vehicles In 2021",2022-01-02,Chinese automotive startup XPeng has shown one...,cleantechnica,https://cleantechnica.com/2022/01/02/xpeng-del...
1,93321,Green Hydrogen: Drop In Bucket Or Big Splash?,2022-01-02,Sinopec has laid plans to build the largest gr...,cleantechnica,https://cleantechnica.com/2022/01/02/its-a-gre...
2,98159,World’ s largest floating PV plant goes online...,2022-01-03,Huaneng Power International has switched on a ...,pv-magazine,https://www.pv-magazine.com/2022/01/03/worlds-...
3,98158,Iran wants to deploy 10 GW of renewables over ...,2022-01-03,According to the Iranian authorities there are...,pv-magazine,https://www.pv-magazine.com/2022/01/03/iran-wa...
4,31128,Eastern Interconnection Power Grid Said ‘ Bein...,2022-01-03,Sign in to get the best natural gas news and d...,naturalgasintel,https://www.naturalgasintel.com/eastern-interc...


In [7]:
# id_list = [42253,
# 22248,
# 105173,
# 31316,
# 43019,
# 31297,
# 31273,
# 31163,
# 31158,
# 31140,
# 126129,
# 6941]
# df_train = df_train[df_train["id"].isin(id_list)]

In [8]:
# df_train

In [9]:
# cleaner = TextCleaner(level="safe")
# data_cleaned = cleaner.clean_text_column(df_train, column="content", new_column="content_cleaned")


In [10]:
cleaner = TextCleaner(level="risky")
data_cleaned = cleaner.clean_text_column(df_train, column="content", new_column="content_cleaned")

In [11]:
# cleaner = TextCleaner(level="dangerous")
# data_cleaned = cleaner.clean_text_column(df_train, column="content", new_column="content_cleaned")

In [12]:
data_cleaned_compare = data_cleaned[["content", "content_cleaned"]].copy()
u.styled_text(data_cleaned_compare.head(), max_width=500)

,content,content_cleaned
0,Chinese automotive startup XPeng has shown one of the most dramatic auto production ramp-ups in hist ory and the good news is it only produces 100% sma rt electric vehicles ( EVs). At a mere 7 years of age and just a few years after launching its first ...,Chinese automotive startup XPeng has shown one of the most dramatic auto production ramp ups in hist ory and the good news is it only produces 100 smar t electric vehicles EVs. At a mere 7 years of age and just a few years after launching its first ve ...
1,Sinopec has laid plans to build the largest green hydrogen production facility in the world but inte rest in fossil-sourced hydrogen continues apace. S kepticism abounds over the green hydrogen field an d yet leading players on the global energy stage s ...,Sinopec has laid plans to build the largest green hydrogen production facility in the world but inte rest in fossil sourced hydrogen continues apace. S kepticism abounds over the green hydrogen field an d yet leading players on the global energy stage s ...
2,Huaneng Power International has switched on a 320 MW floating PV array in China’ s Shandong province . It deployed the plant in two phases on a reservo ir near its 2.65 GW Dezhou thermal power station. Huaneng Power International ( HPI) has completed t ...,Huaneng Power International has switched on a 320 MW floating PV array in China' s Shandong province . It deployed the plant in two phases on a reservo ir near its 2.65 GW Dezhou thermal power station. Huaneng Power International HPI has completed the ...
3,According to the Iranian authorities there are cur rently more than 80GW of renewable energy projects that were submitted by private investors for revi ew. Iran had around 414MW of installed solar power at the end of 2020. The Iranian Energy Ministry a ...,According to the Iranian authorities there are cur rently more than 80GW of renewable energy projects that were submitted by private investors for revi ew. Iran had around 414MW of installed solar power at the end of 2020. The Iranian Energy Ministry a ...
4,Sign in to get the best natural gas news and data. Follow the topics you want and receive the daily emails. Your email address * Your password * Remem ber me Continue Reset password Featured Content Ne ws & Data Services Client Support Daily GPI Infras ...,"Follow the topics you want and However the plann ers known as the Eastern Interconnection Planning Collaborative EIPC added a cautionary note. EIPC' s Zach Smith executive committee chairman said pl anners concluded that "" the Eastern Interconnectio ..."


In [13]:
u.print_full_text(data_cleaned.head(1))

,id,title,date,content,domain,url,content_cleaned
0,93320,"XPeng Delivered ~100,000 Vehicles In 2021",2022-01-02 00:00:00,Chinese automotive startup XPeng has shown one of the most dramatic auto production ramp-ups in history and the good news is it only produces 100% smart electric vehicles ( EVs). At a mere 7 years of age and just a few years after launching its first vehicle ( the XPeng G3 went on sales in December 2018) XPeng has wrapped up a year of sales totaling almost 100 000. In 2021 XPeng delivered 98 155 smart EVs a 263% year-over-year increase. In December 16 000 vehicles were delivered a 181% year-over-year increase compared to December 2020. There were 41 751 vehicles delivered in Q4 2021 which was a 222% year-over-year increase. Reinforcing how impressive the 98 155 delivery figure for 2021 is at the end of the year XPeng reached just 137 953 cumulative deliveries. All the more impressive XPeng’ s monthly delivery target is 15 000 vehicles and December was the second month in a row that was surpassed — and that’ s despite the global automotive chip shortage and other supply chain challenges. In the case of the P7 deliveries were up 102%. GS deliveries were up 75%. The P5 was not on the market one year ago but its sales did increase 134% month over month. XPeng also noted that its broader charging and sales network in China has been growing “ rapidly. ” It had “ 661 branded supercharging stations across 228 cities and 311 physical retail stores in operation across 121 cities as of the end of November 2021. ” All in all XPeng’ s growth seems to be rolling out as the company hoped and planned or even better. It’ s quite an uplifting sight to see another pure EV company scaling up production rapidly and achieving great sales success at a young age. Of course the reference company need not be named and we need not compare the two or pretend XPeng should exactly follow Tesla’ s ( whoops) path. However the point is that we are getting more examples of pure EV companies achieving great successes. Zach is tryin ' to help society help itself one word at a time. He spends most of his time here on CleanTechnica as its director chief editor and CEO. Zach is recognized globally as an electric vehicle solar energy and energy storage expert. He has presented about cleantech at conferences in India the UAE Ukraine Poland Germany the Netherlands the USA Canada and Curaçao. Zach has long-term investments in Tesla [ TSLA ] NIO [ NIO ] Xpeng [ XPEV ] Ford [ F ] ChargePoint [ CHPT ] Amazon [ AMZN ] Piedmont Lithium [ PLL ] Lithium Americas [ LAC ] Albemarle Corporation [ ALB ] Nouveau Monde Graphite [ NMGRF ] Talon Metals [ TLOFF ] Arclight Clean Transition Corp [ ACTC ] and Starbucks [ SBUX ]. But he does not offer ( explicitly or implicitly) investment advice of any sort. Advertise with CleanTechnica to get your company in front of millions of monthly readers. Rethink Energy has made the prediction that some countries’ entire automotive fleets will be electric within the next decade. Policy changes in various countries... Tesla hits the UK with its strongest ever May helping lift the plugin electric vehicle market share to 23.1% up from 18.3% year on... The auto market in Germany saw plugin EVs take 22.9% share in May 2023 down from 25.3% year on year. Full electrics gained share ... Recently we did a story about the 2023 NADA convention and goat roping in Dallas where Slate correspondent Alexander Sammon got a chance to... Copyright © 2023 CleanTechnica. The content produced by this site is for entertainment purposes only. Opinions and comments published on this site may not be sanctioned by and do not necessarily represent the views of CleanTechnica its owners sponsors affiliates or subsidiaries.,cleantechnica,https://cleantechnica.com/2022/01/02/xpeng-delivered-100000-vehicles-in-2021/,"Chinese automotive startup XPeng has shown one of the most dramatic auto production ramp ups in history and the good news is it only produces 10

In [14]:
# data_cleaned = u.clean_url_column(data_cleaned, 'url')

In [15]:
u.print_full_text(df = data_cleaned[data_cleaned['id'] == 105173])


In [16]:
file_name_test = 'cleantech_rag_evaluation_data_2024-09-20.csv'
# full_path = os.path.join(file_path, file_name_train)

column_names = ["example_id", "question_id", "question", "relevant_text", "answer", "article_url"]
# df_train = pd.read_csv(full_path, delimiter=",", names=column_names, skiprows=1, encoding="utf-8")
# df_train = pd.read_csv(file_path+file_name_test, delimiter=",", names=column_names, skiprows=1, encoding="utf-8")
df_test = pd.read_csv(file_path + file_name_test, delimiter=";", names=column_names, skiprows=1, encoding="utf-8")


In [17]:
# u.print_full_text(df_test)

In [18]:
df_pv_magazine = df_train[df_train['url'] == 'https://www.pv-magazine.com/2023/04/08/high-time-for-solar/#comments']

In [19]:
u.print_full_text(df_pv_magazine)

In [20]:
def check_target_url_in_train_df(target_df: pd.DataFrame, train_df: pd.DataFrame) -> pd.DataFrame:
    url_in_train_df = set(train_df['url'])
    target_df['url_in_train_df'] = target_df['article_url'].isin(url_in_train_df)
    return target_df

In [21]:
url_check = check_target_url_in_train_df(df_test, data_cleaned)
url_check = url_check[url_check['url_in_train_df'] == False]
u.print_full_text(url_check)

,example_id,question_id,question,relevant_text,answer,article_url,url_in_train_df
0,1,1,What is the innovation behind Leclanché's new method to produce lithium-ion batteries?,"Leclanché said it has developed an environmentally friendly way to produce lithium-ion (Li-ion) batteries. It has replaced highly toxic organic solvents, commonly used in the production process, with a water-based process to make nickel-manganese-cobalt-aluminium cathodes (NMCA).",Leclanché's innovation is using a water-based process instead of highly toxic organic solvents to produce nickel-manganese-cobalt-aluminium cathodes for lithium-ion batteries.,https://www.sgvoice.net/strategy/technology/23971/leclanches-new-disruptive-battery-boosts-energy-density/,False
1,2,2,What is the EU’s Green Deal Industrial Plan?,"The Green Deal Industrial Plan is a bid by the EU to make its net zero industry more competitive and to accelerate its transition to net zero. It intends to support the expansion of European manufacturing of technologies, goods and services needed to achieve its climate targets.","The EU’s Green Deal Industrial Plan aims to enhance the competitiveness of its net zero industry and accelerate the transition to net zero by supporting the expansion of European manufacturing of technologies, goods, and services necessary to meet climate targets.",https://www.sgvoice.net/policy/25396/eu-seeks-competitive-boost-with-green-deal-industrial-plan/,False
3,4,3,What are the four focus areas of the EU's Green Deal Industrial Plan?,"The new plan is fundamentally focused on four areas, or pillars: the regulatory environment, access to finance, enhancing skills, and improving supply chain resilience. It also builds on other initiatives, such as REPowerEU, as well as the strength of the single market.","The four focus areas of the EU's Green Deal Industrial Plan are the regulatory environment, access to finance, enhancing skills, and improving supply chain resilience.",https://www.sgvoice.net/policy/25396/eu-seeks-competitive-boost-with-green-deal-industrial-plan/,False
22,23,21,Which has the higher absorption coefficient for wavelengths above 500m - amorphous germanium or amorphous silicon?,We chose amorphous germanium instead of amorphous silicon as absorber material because of its higher absorption coefficient for wavelengths above 500 nm.,amorphous germanium,https://www.pv-magazine.com/2021/01/15/germanium-based-solar-cell-tech-for-agrivoltaics/#respond,False


In [22]:
# print(df_train[df_train['url'].str.startswith('https://www.pv-magazine.com/2021')])

In [23]:
# url_check.to_csv(file_path+'data_processed/target_urls_not_in_train_df.csv', index=False, sep=';')

In [24]:
# # save evaluation data without false urls
# url_check = check_target_url_in_train_df(df_test, data_cleaned)
# cleantech_rag_evaluation_data = url_check[url_check['url_in_train_df'] == True]

# cleantech_rag_evaluation_data = cleantech_rag_evaluation_data.drop(columns=['url_in_train_df'])

# # save as parquet file
# cleantech_rag_evaluation_data.to_parquet('../data_mc1/data_processed/cleantech_rag_evaluation_data.parquet', index=False)

# read parquet file
cleantech_rag_evaluation_data = pd.read_parquet('../data_mc1/data_processed/cleantech_rag_evaluation_data.parquet')
cleantech_rag_evaluation_data.head()

,example_id,question_id,question,relevant_text,answer,article_url
0,3,2,What is the EU’s Green Deal Industrial Plan?,The European counterpart to the US Inflation R...,The EU’s Green Deal Industrial Plan aims to en...,https://www.pv-magazine.com/2023/02/02/europea...
1,5,4,When did the cooperation between GM and Honda ...,What caught our eye was a new hookup between G...,July 2013,https://cleantechnica.com/2023/05/08/general-m...
2,6,5,Did Colgate-Palmolive enter into PPA agreement...,"Scout Clean Energy, a Colorado-based renewable...",yes,https://solarindustrymag.com/scout-and-colgate...
3,7,6,What is the status of ZeroAvia's hydrogen fuel...,"In December, the US startup ZeroAvia announced...",ZeroAvia's hydrogen fuel cell electric aircraf...,https://cleantechnica.com/2023/01/02/the-wait-...
4,8,7,"What is the ""Danger Season""?",As spring turns to summer and the days warm up...,"The ""Danger Season"" is the period in the North...",https://cleantechnica.com/2023/05/15/what-does...
